In [6]:
%reset -f

import spacy
from spacy.tokens import Doc
import re
import PyPDF2
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
from transformers import EarlyStoppingCallback


from sklearn.metrics import f1_score

In [7]:
test_df = pd.read_csv('balanced dataset/test_dataset_208_new.csv')
# test_df = pd.read_csv('balanced dataset/test_dataset_208_new.csv')

# test_df = pd.read_csv('redacted_training_data.csv', sep=';')
# test_df = test_df.rename(columns={'is_goal': 'label'})
# def func(s):
#     if s == 'Y':
#         return 1
#     else:
#         return 0

# test_df['label'] = test_df['label'].apply(func)



action_statements = list(test_df['sentence'])

In [8]:
# Fourth stage - Inference
base_model = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(base_model)
action_statements_tokenized = tokenizer(action_statements, padding=True, truncation=True, max_length=512)

class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

test_dataset = Dataset(action_statements_tokenized)

model_path = 'mar11/checkpoint-180'
model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=2)
test_trainer = Trainer(model)


raw_pred, _, _ = test_trainer.predict(test_dataset)
y_pred = np.argmax(raw_pred, axis=1)

test_df['prediction'] = pd.Series(y_pred)
# y_pred = pd.Series(y_pred)
# goal_indices = list(y_pred[y_pred==1].index)

# c = 1
# for i in goal_indices:
#     print(f'Goal{c}')
#     print(action_statements[i])
#     print('')
#     c += 1

loading configuration file config.json from cache at /Users/shubham/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file vocab.txt from cache at /Users/shubham/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/vocab.t

In [9]:
confusion_matrix = pd.crosstab(test_df['label'], test_df['prediction'], rownames=['Actual'], colnames=['Predicted'])
print (confusion_matrix)

Predicted   0   1
Actual           
0          94  10
1          14  90


In [10]:


f1_score(test_df['label'], test_df['prediction'])

0.8823529411764707